# 메소드와 객체 정의
## 휴장일 파일의 PATH 잡아줘야함 

In [1]:
import sys
import datetime


CLOSE_DAY_FILE = r'C:\Users\jang\PycharmProjects\close_day.txt'


def get_close_date_list():
    # 휴장일을 텍스트파일로부터 받아옴
    # 양식 : YYYYMMDD
    # 20140101
    # 20140505
    # 20140815
    try:
        with open(CLOSE_DAY_FILE, 'r') as f:
            close_date_list = []
            for line in f.readlines():
                date = line.split("\n")[0]
                close_date_list.append(date)
    except FileNotFoundError:
        print('휴장일 데이터 파일이 없음')
        sys.exit(-1)
    return tuple(close_date_list)


def get_trading_days_list(min_year=2010, size=4000):
    # 임의로 2010년 부터 시작함
    base = datetime.date(min_year, 1, 1)
    date_list = [base + datetime.timedelta(days=x) for x in range(0, size)]

    # 주말제외한 리스트
    week_days_list = []
    for date in date_list:
        if date.weekday() not in [5, 6]:  # 5:토요일 6:일요일
            week_days_list.append(date)

    # 휴장일 제외
    close_date_list = get_close_date_list()
    close_date_time_list = []
    for close_date in close_date_list:
        close_date_time_list.append(datetime.datetime.strptime(close_date, '%Y%m%d').date())
    trading_days_list = list(set(week_days_list) - set(close_date_time_list))
    trading_days_list.sort()
    return tuple(trading_days_list)


class TradingDay:
    trading_days_list = get_trading_days_list()

    def __init__(self, target):
        self.target_date = datetime.datetime.strptime(str(target), '%Y%m%d').date()
        if self.target_date not in TradingDay.trading_days_list:
            raise ValueError(target, '는 휴장일')

    def get_target_date(self):
        return self.target_date

    def get_target_date_str_t(self):
        return self.target_date.strftime('%Y%m%d')

    def get_target_date_int_t(self):
        return int(self.get_target_date_str_t())

    def set_target_date(self, target):
        temp_date = datetime.datetime.strptime(str(target), '%Y%m%d').date()
        if temp_date not in self.trading_days_list:
            raise IndexError(temp_date, '는 휴장일')
        else:
            self.target_date = temp_date

    def delta(self, days):
        index = TradingDay.trading_days_list.index(self.get_target_date())
        return TradingDay.trading_days_list[index + days]

    def delta_str_t(self, days):
        return self.delta(days).strftime('%Y%m%d')

    def delta_int_t(self, days):
        return int(self.delta_str_t(days))

    @staticmethod
    def trading_days_between_date(cur_date, target_date):
        """
        cur_date, target_date : int 타입
        cur_date 가 target_date 로 부터 몇 거래일인지 리턴해줌
        :return:
        """
        cur_date = datetime.datetime.strptime(str(cur_date), '%Y%m%d').date()
        target_date = datetime.datetime.strptime(str(target_date), '%Y%m%d').date()
        cur_date_index = TradingDay.trading_days_list.index(cur_date)
        target_date_index = TradingDay.trading_days_list.index(target_date)
        return cur_date_index - target_date_index

    @staticmethod
    def get_trading_day_list(start_date, end_date):
        start_date = datetime.datetime.strptime(str(start_date), '%Y%m%d').date()
        end_date = datetime.datetime.strptime(str(end_date), '%Y%m%d').date()

        if start_date not in TradingDay.trading_days_list:
            raise ValueError(start_date, '휴장일')
        elif end_date not in TradingDay.trading_days_list:
            raise ValueError(end_date, '휴장일')

        std_idx = TradingDay.trading_days_list.index(start_date)
        end_idx = TradingDay.trading_days_list.index(end_date)
        return [int(date.strftime('%Y%m%d')) for date in TradingDay.trading_days_list[std_idx:end_idx + 1]]

    @staticmethod
    def trading_days_by_year(year):
        date_list = []
        for date in TradingDay.trading_days_list:
            if date.year == year:
                date_list.append(int(date.strftime('%Y%m%d')))
        return date_list

    @staticmethod
    def trading_days_by_year_n_month(year, month):
        """
        year, month 인자로 받아
        거래일 리스트 리턴
        """
        date_list = []
        for date in TradingDay.trading_days_list:
            if date.year == year and date.month == month:
                date_list.append(int(date.strftime('%Y%m%d')))
        return date_list

    @staticmethod
    def expiration_date(year, month):
        """
        만기일은 매달 두번째 목요일
        연도와 달을 인자로 받아
        만기일을 리턴하는 함수
        """
        first_thursday = None
        for i in range(1, 8):
            date = datetime.datetime(year, month, i).date()
            if date.weekday() == 3:
                first_thursday = date
        second_thursday = int((first_thursday + datetime.timedelta(days=7)).strftime('%Y%m%d'))

        trading_days = TradingDay.trading_days_by_year_n_month(year, month)
        if second_thursday in trading_days:
            return second_thursday
        else:
            trading_days.reverse()
            for date in trading_days:
                if date < second_thursday:
                    return date
    
    @staticmethod
    def magnet(date, flag):
        """
        flag:: before:0, after:1
        """
        date = datetime.datetime.strptime(str(date), '%Y%m%d').date()
        idx = 0
        for trading_day in TradingDay.trading_days_list:
            if date <= trading_day:
                break
            idx += 1
        if date == TradingDay.trading_days_list[idx]:
            return int(TradingDay.trading_days_list[idx].strftime('%Y%m%d'))
        else:
            if flag == 0:
                return int(TradingDay.trading_days_list[idx-1].strftime('%Y%m%d'))
            else:
                return int(TradingDay.trading_days_list[idx].strftime('%Y%m%d'))


# TradingDay 객체 생성
## 인자는 기준으로 설정할 거래일 YYYYMMDD

In [2]:
td = TradingDay(20171124)

### 기준일 출력

In [3]:
td.get_target_date()

datetime.date(2017, 11, 24)

In [4]:
td.get_target_date_int_t()

20171124

In [5]:
td.get_target_date_str_t()

'20171124'

### 기준일로 부터 t 거래일 떨어진 거래일의 날짜

In [6]:
td.delta(10)

datetime.date(2017, 12, 8)

In [7]:
td.delta_int_t(10)

20171208

In [8]:
td.delta_str_t(10)

'20171208'

In [9]:
td.delta(-20)

datetime.date(2017, 10, 27)

In [10]:
td.delta_int_t(-20)

20171027

In [11]:
td.delta_str_t(-20)

'20171027'

### 거래일과 거래일 사이의 거래일

In [12]:
TradingDay.trading_days_between_date(20171124, 20171101)

17

### 시작일과 종료일을 인자로 받아 거래일 리스트 리턴

In [13]:
TradingDay.get_trading_day_list(20171101, 20171124)

[20171101,
 20171102,
 20171103,
 20171106,
 20171107,
 20171108,
 20171109,
 20171110,
 20171113,
 20171114,
 20171115,
 20171116,
 20171117,
 20171120,
 20171121,
 20171122,
 20171123,
 20171124]

###  해당하는 연도의 거래일

In [14]:
TradingDay.trading_days_by_year(2017)

[20170102,
 20170103,
 20170104,
 20170105,
 20170106,
 20170109,
 20170110,
 20170111,
 20170112,
 20170113,
 20170116,
 20170117,
 20170118,
 20170119,
 20170120,
 20170123,
 20170124,
 20170125,
 20170126,
 20170131,
 20170201,
 20170202,
 20170203,
 20170206,
 20170207,
 20170208,
 20170209,
 20170210,
 20170213,
 20170214,
 20170215,
 20170216,
 20170217,
 20170220,
 20170221,
 20170222,
 20170223,
 20170224,
 20170227,
 20170228,
 20170302,
 20170303,
 20170306,
 20170307,
 20170308,
 20170309,
 20170310,
 20170313,
 20170314,
 20170315,
 20170316,
 20170317,
 20170320,
 20170321,
 20170322,
 20170323,
 20170324,
 20170327,
 20170328,
 20170329,
 20170330,
 20170331,
 20170403,
 20170404,
 20170405,
 20170406,
 20170407,
 20170410,
 20170411,
 20170412,
 20170413,
 20170414,
 20170417,
 20170418,
 20170419,
 20170420,
 20170421,
 20170424,
 20170425,
 20170426,
 20170427,
 20170428,
 20170502,
 20170504,
 20170508,
 20170510,
 20170511,
 20170512,
 20170515,
 20170516,
 20170517,

### 해당하는 월의 거래일

In [15]:
TradingDay.trading_days_by_year_n_month(2017, 11)

[20171101,
 20171102,
 20171103,
 20171106,
 20171107,
 20171108,
 20171109,
 20171110,
 20171113,
 20171114,
 20171115,
 20171116,
 20171117,
 20171120,
 20171121,
 20171122,
 20171123,
 20171124,
 20171127,
 20171128,
 20171129,
 20171130]

### 만기일

In [16]:
TradingDay.expiration_date(2017, 11)

20171109

### 해당날짜가 거래일이 아닌경우 근접한 거래일 리턴

In [17]:
TradingDay.magnet(20171125, 0)

20171124

In [18]:
TradingDay.magnet(20171125, 1)

20171127